In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from keras.models import load_model
from tensorflow_addons.optimizers import RectifiedAdam
import tensorflow as tf
from tqdm import tqdm

tf.keras.optimizers.RectifiedAdam = RectifiedAdam
import janestreet
env = janestreet.make_env()
models = []
FOLDS = 5
SEED = 1111
for f in range(FOLDS):
    model=load_model(f'../input/simple-mlp/model_{SEED}_{f}_finetune.h5')
    models.append(model)
print('model is done')

train = pd.read_csv('../input/jane-street-market-prediction/train.csv')
features = [c for c in train.columns if 'feature' in c]
f_mean = np.mean(train[features[1:]].values,axis=0)

f = np.median
models = models[-2:]
th = 0.5
for (test_df, pred_df) in tqdm(env.iter_test()):
    if test_df['weight'].item() > 0:
        x_tt = test_df.loc[:, features].values
        if np.isnan(x_tt[:, 1:].sum()):
            x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean
        pred = np.mean([model(x_tt, training = False).numpy() for model in models],axis=0)
        pred = f(pred)
        pred_df.action = np.where(pred >= th, 1, 0).astype(int)
    else:
        pred_df.action = 0
    env.predict(pred_df)